In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Embedding, Bidirectional, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [4]:
tokenizer = Tokenizer()

In [5]:
with open("lanigan ball.txt", encoding="utf-8") as conteudo:
    data = conteudo.read()

corpus = data.lower().split("\n")

In [6]:
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0] # tokeniza a lista

    for i in range(1, len(token_list)): # cria n-gramas
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [8]:
max_sequence_length = max([len(x) for x in input_sequences])

In [11]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding="pre"))

In [13]:
# slicing
xs = input_sequences[:, :-1]
labels = input_sequences[:, -1]

In [14]:
# one-hot encoding
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [15]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation="softmax"))
adam = Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])
history = model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100
10/10 [==============================] - 4s 26ms/step - loss: 5.2101 - accuracy: 0.0411
Epoch 2/100
10/10 [==============================] - 0s 35ms/step - loss: 4.7842 - accuracy: 0.0759
Epoch 3/100
10/10 [==============================] - 0s 27ms/step - loss: 4.2785 - accuracy: 0.0949
Epoch 4/100
10/10 [==============================] - 0s 25ms/step - loss: 3.6189 - accuracy: 0.1551
Epoch 5/100
10/10 [==============================] - 0s 33ms/step - loss: 2.9927 - accuracy: 0.2405
Epoch 6/100
10/10 [==============================] - 0s 24ms/step - loss: 2.1849 - accuracy: 0.3861
Epoch 7/100
10/10 [==============================] - 0s 26ms/step - loss: 1.5135 - accuracy: 0.5918
Epoch 8/100
10/10 [==============================] - 0s 25ms/step - loss: 1.0278 - accuracy: 0.7089
Epoch 9/100
10/10 [==============================] - 0s 25ms/step - loss: 0.6813 - accuracy: 0.8133
Epoch 10/100
10/10 [==============================] - 0s 27ms/step - loss: 0.4521 - accuracy: 0.8861

In [ ]:
seed_text = 'I made a poetry machine'
next_words = 20


for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding="pre")
    predicted = model.predict(token_list, verbose=0)

    # index da maior probabilidade
    predicted_index = np.argmax(predicted, axis=-1)[0]

    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break

    # atualiza o texto com a nova palavra prevista
    seed_text += " " + output_word

print(seed_text)

I made a poetry machine faint and cried ‘holy murder ’ a man again left him a farm and ten acres of ground ground ground
